In [1]:
import pandas as pd

from transformers import AutoTokenizer
from transformers import BertTokenizerFast
from transformers import BertTokenizer
from transformers import RobertaTokenizerFast
#from transformers import ROBERTAtokenizer
#from transformers import XLNETtokenizer
from mlxtend.text import tokenizer_emoticons
from nltk.tokenize.casual import TweetTokenizer

import re, string
import emoji

In [2]:
df = pd.read_csv('/kaggle/input/masculinity-saturday-twitter-sentiment-analysis/twitter_sentiment_analysis_masculinity_saturday_data_annotated.csv', index_col=0)

In [3]:
df.head()

,tweet_created_at,text,Positive,Negative,Neutral,Compound,Sentiment
0,2022-12-30 22:37:46+00:00,Men who have refused to listen to/attend @amer...,0.000,0.073,0.927,-0.2960,Negative
1,2022-12-30 22:30:58+00:00,@OvOBrezzzy Take amerix next,0.000,0.000,1.000,0.0000,Neutral
2,2022-12-30 22:18:00+00:00,@AyodeleYo11 @amerix Nah I’m good my masculini...,0.493,0.068,0.439,0.8271,Positive
3,2022-12-30 20:42:54+00:00,@Amerix_DontSimp There is no need for dating i...,0.000,0.180,0.820,-0.2960,Negative
4,2022-12-30 20:32:23+00:00,@DyeAnna7 @amerix @kibe From what cos nothing ...,0.110,0.000,0.890,0.6908,Positive


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 248738 entries, 0 to 248738
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   tweet_created_at  248738 non-null  object 
 1   text              248738 non-null  object 
 2   Positive          248738 non-null  float64
 3   Negative          248738 non-null  float64
 4   Neutral           248738 non-null  float64
 5   Compound          248738 non-null  float64
 6   Sentiment         248738 non-null  object 
dtypes: float64(4), object(3)
memory usage: 15.2+ MB


In [5]:
df.columns

Index(['tweet_created_at', 'text', 'Positive', 'Negative', 'Neutral',
       'Compound', 'Sentiment'],
      dtype='object')

In [6]:
df = df[['tweet_created_at', 'text', 'Sentiment']]

In [7]:
df.head()

,tweet_created_at,text,Sentiment
0,2022-12-30 22:37:46+00:00,Men who have refused to listen to/attend @amer...,Negative
1,2022-12-30 22:30:58+00:00,@OvOBrezzzy Take amerix next,Neutral
2,2022-12-30 22:18:00+00:00,@AyodeleYo11 @amerix Nah I’m good my masculini...,Positive
3,2022-12-30 20:42:54+00:00,@Amerix_DontSimp There is no need for dating i...,Negative
4,2022-12-30 20:32:23+00:00,@DyeAnna7 @amerix @kibe From what cos nothing ...,Positive


In [8]:
df['text'].head(20)

0     Men who have refused to listen to/attend @amer...
1                          @OvOBrezzzy Take amerix next
2     @AyodeleYo11 @amerix Nah I’m good my masculini...
3     @Amerix_DontSimp There is no need for dating i...
4     @DyeAnna7 @amerix @kibe From what cos nothing ...
5     @salim_the_don @MwendiaJnr I see you unblocked...
6     @german_kenyan @ML_Philosophy 🤣 yes they are a...
7     @PadreMusa Amerix once said the alcohol menace...
8     @DodoPicard You're in Eastafrica, he probably ...
9                      @HBK_ACTIVIST @amerix @kibe Heal
10                           Amerix student of the year
11    @sholla_saddam These are notes from @amerix's ...
12    @amerix I appreciate the intent of this, but I...
13    @Mynames_Jefff @Biancawamu2 @amerix Wouldn't b...
14    @sharbabababa @Biancawamu2 could be @amerix wife🤷
15    @DyeAnna7 @amerix @kibe Was raised by toxic co...
16    @butdarkacademia @purevibeezz good thing is An...
17    @amerix The system needs you sick and depr

In [9]:
df.tail(20)

,tweet_created_at,text,Sentiment
248719,2020-01-03 16:00:23+00:00,#FoodFriday means tasty snacks throughout the ...,Neutral
248720,2020-01-03 15:32:14+00:00,"F U E L E D F R I D A Y\nLooking for a fun, co...",Positive
248721,2020-01-03 15:15:13+00:00,Kummer reminds you to stay hydrated #foodfrida...,Neutral
248722,2020-01-03 15:14:57+00:00,Commercial #Kitchens need attention at a great...,Positive
248723,2020-01-03 15:11:01+00:00,Mary's Meals is changing lives in Zimbabwe htt...,Neutral
248724,2020-01-03 15:00:48+00:00,Will meat alternatives continue gaining popula...,Positive
248725,2020-01-03 14:29:10+00:00,These are our top five healthy recipes that yo...,Positive
248726,2020-01-03 14:01:57+00:00,Burned out on cooking after the holidays? Here...,Positive
248727,2020-01-03 14:00:38+00:00,Will you eat lettuce that is not grown in soil...,Neutral
248728,2020-01-03 13:49:24+00:00,#FoodFriday: Is it possible to treat things li...,Negative


In [10]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
tokenizer_1 = AutoTokenizer.from_pretrained('bert-base-uncased')

In [12]:
emoji_list = list(emoji.EMOJI_DATA.keys())

cnt = 0

for e in emoji_list:
    tokenized = tokenizer.decode(tokenizer.encode(e)).strip("</s>").strip()
    if e not in tokenized:
        cnt += 1
print(f"{cnt/len(emoji_list)*100}% of the emojis are not identified by this tokenizer.")

99.80984576378619% of the emojis are not identified by this tokenizer.


In [13]:
emoji_list = list(emoji.EMOJI_DATA.keys())

cnt = 0

for e in emoji_list:
    tokenized = tokenizer_1.decode(tokenizer_1.encode(e)).strip("</s>").strip()
    if e not in tokenized:
        cnt += 1
print(f"{cnt/len(emoji_list)*100}% of the emojis are not identified by this tokenizer.")

99.80984576378619% of the emojis are not identified by this tokenizer.


In [14]:
tokenizer_3 = BertTokenizer.from_pretrained('bert-base-uncased')

In [15]:
emoji_list = list(emoji.EMOJI_DATA.keys())

cnt = 0

for e in emoji_list:
    tokenized = tokenizer_3.decode(tokenizer_3.encode(e)).strip("</s>").strip()
    if e not in tokenized:
        cnt += 1
print(f"{cnt/len(emoji_list)*100}% of the emojis are not identified by this tokenizer.")

99.80984576378619% of the emojis are not identified by this tokenizer.


In [16]:
tokenizer_4 = AutoTokenizer.from_pretrained("openai-gpt")

Downloading:   0%|          | 0.00/656 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/797k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/448k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

In [17]:
emoji_list = list(emoji.EMOJI_DATA.keys())

cnt = 0

for e in emoji_list:
    tokenized = tokenizer_4.decode(tokenizer_4.encode(e)).strip("</s>").strip()
    if e not in tokenized:
        cnt += 1
print(f"{cnt/len(emoji_list)*100}% of the emojis are not identified by this tokenizer.")

99.80984576378619% of the emojis are not identified by this tokenizer.


In [18]:
def emoji2description(text):

    return emoji.replace_emoji(text, replace=lambda chars, data_dict: ' '.join(data_dict['en'].split('_')).strip(':'))

In [19]:
def emoji2concat_emoji(text):
    emoji_list = emoji.emoji_list(text)
    ret = emoji.replace_emoji(text, replace='').strip()
    for json in emoji_list:
        this_emoji = json['emoji']
        ret += ' ' + this_emoji
    return ret

In [20]:
def emoji2concat_description(text):
    emoji_list = emoji.emoji_list(text)
    ret = emoji.replace_emoji(text, replace='').strip()
    for json in emoji_list:
        this_desc = ' '.join(emoji.EMOJI_DATA[json['emoji']]['en'].split('_')).strip(':')
        ret += ' ' + this_desc
    return ret

In [21]:
texts_new = []

for t in df.text:
    texts_new.append(
        emoji2description(emoji2concat_emoji(emoji2concat_description((t))))
    )

In [22]:
df['text_clean'] = texts_new

In [23]:
df['text'].tail(20).values

array(['#FoodFriday means tasty snacks throughout the day!\n😋💙🥄🍫\n📷 : @flexible_nati\n#haveaplant https://t.co/pNSAHV2hDh',
       'F U E L E D F R I D A Y\nLooking for a fun, colorful and healthy recipe to kick off the new year? Well, check out this delicious and beautiful dish!\n・・・\nEggs baked in Veggie Nests. Head over to our insta for the recipe. 🤤\n\n#fueledfriday #foodfriday #runinrabbit #borntorunfree https://t.co/2Fwez5uYb2',
       'Kummer reminds you to stay hydrated #foodfriday https://t.co/rkZmatd97f',
       'Commercial #Kitchens need attention at a great price! 🦏 Rhino Shine has a solid plan unlike #JasonGarrett future 🤷🏻\u200d♂️. #Dallas #winning #coupons #restaurant #hood #cleaning #foodfriday #trucks https://t.co/U0V5BO33uo',
       "Mary's Meals is changing lives in Zimbabwe https://t.co/bioEZAt47N #feelgoodfriday #FoodFriday",
       'Will meat alternatives continue gaining popularity in 2020? https://t.co/wzvi1XOBmj via @YourMorning ht  @LetsTalkScience 🥬🍔🤑 #food #

In [24]:
df['text_clean'].tail(20).values

array(['#FoodFriday means tasty snacks throughout the day!\n\n : @flexible_nati\n#haveaplant https://t.co/pNSAHV2hDh face savoring food blue heart spoon chocolate bar camera',
       'F U E L E D F R I D A Y\nLooking for a fun, colorful and healthy recipe to kick off the new year? Well, check out this delicious and beautiful dish!\n・・・\nEggs baked in Veggie Nests. Head over to our insta for the recipe. \n\n#fueledfriday #foodfriday #runinrabbit #borntorunfree https://t.co/2Fwez5uYb2 drooling face',
       'Kummer reminds you to stay hydrated #foodfriday https://t.co/rkZmatd97f',
       'Commercial #Kitchens need attention at a great price!  Rhino Shine has a solid plan unlike #JasonGarrett future . #Dallas #winning #coupons #restaurant #hood #cleaning #foodfriday #trucks https://t.co/U0V5BO33uo rhinoceros man shrugging light skin tone',
       "Mary's Meals is changing lives in Zimbabwe https://t.co/bioEZAt47N #feelgoodfriday #FoodFriday",
       'Will meat alternatives continue gainin

In [25]:
# remove multiple spaces

def remove_mult_spaces(text): 
    return re.sub("\s\s+" , " ", text)

In [26]:
#Remove punctuations, links, mentions and \r\n new line characters

def strip_all_entities(text): 
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    return text

In [27]:
#clean hashtags at the end of the sentence, and keep those in the middle of the sentence by removing just the # symbol

def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) #remove hashtags symbol from words in the middle of the sentence
    return new_tweet2

In [28]:
#Filter special characters such as & and $ present in some words

def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

In [29]:
deeper_clean = []

for t in df.text_clean:
    deeper_clean.append(
        remove_mult_spaces(
            filter_chars(
                clean_hashtags(
                    strip_all_entities(
                        (t)
                    ))))
    )